In [35]:
include("../code/exact_enum.jl")
include("../code/matrix_generator.jl")
include("../code/bp_full.jl")
include("../../telegram/notifications.jl")
using Base.Threads
nthreads()

22

In [36]:
Rs = [0.25]
# ns = [56, 68, 80, 88, 96, 100]
ns = [56, 68, 80, 88, 92, 96, 100]
navg = 5000 .- ns
nsources = 100

d = [[fill(NaN, navg[i]) for _ in Rs] for i in eachindex(ns)]
nz = [[fill(NaN, navg[i]) for _ in Rs] for i in eachindex(ns)]
wef_src = [[[zeros(Int, ns[i]) for _ in 1:navg[i]] for _ in Rs] for i in eachindex(ns)]
wef_zero = [[[zeros(Int, ns[i]) for _ in 1:navg[i]] for _ in Rs] for i in eachindex(ns)]
t = fill(NaN, length(ns));

In [37]:
for (a,n) in enumerate(ns)
    @showprogress for (i,R) in enumerate(Rs)
        println("Rate $i of ", length(Rs))
        m = round(Int, n*(1-R))
        f3 = 1-3R
        Λ = [0,1-f3,f3]
        K = [0, 0, 1]
        nedges = 3m 
        @show n, R
        H = permutedims(ldpc_matrix(n,m,nedges,Λ,K))
    end
end

Rate 1 of 1
(n, R) = (56, 0.25)
Rate 1 of 1
(n, R) = (68, 0.25)
Rate 1 of 1
(n, R) = (80, 0.25)
Rate 1 of 1
(n, R) = (88, 0.25)
Rate 1 of 1
(n, R) = (92, 0.25)
Rate 1 of 1
(n, R) = (96, 0.25)
Rate 1 of 1
(n, R) = (100, 0.25)


In [ ]:
@telegram "Exact enumeration single rate" for (a,n) in enumerate(ns)
    println("### Size $a of ", length(ns))
    tt = @timed begin
        @showprogress for (i,R) in enumerate(Rs)
            println("Rate $i of ", length(Rs))
            m = round(Int, n*(1-R))
            f3 = 1-3R
            Λ = [0,1-f3,f3]
            K = [0, 0, 1]
            nedges = 3m
            @threads for j in 1:navg[a]  
                H = permutedims(ldpc_matrix(n,m,nedges,Λ,K))
                s = [bitrand(n) for _ in nsources]
                B, indep = findbasis_slow(BitMatrix(H))
                rtrue = size(B,2) / size(B,1)
                if rtrue == R
                    h0, h, mins = exact_wef(B, s)
                    wef_zero[a][i][j] = h0
                    wef_src[a][i][j] = h[1]
                    d[a][i][j] = mean(mins)
                end
            end
        end
    end
    t[a] = tt.time
    d
end

┌ Warning: Assignment to `m` in soft scope is ambiguous because a global variable by the same name exists: `m` will be treated as a new local. Disambiguate by using `local m` to suppress this warning or `global m` to assign to the existing global variable.
└ @ nothing In[38]:6
┌ Warning: Assignment to `f3` in soft scope is ambiguous because a global variable by the same name exists: `f3` will be treated as a new local. Disambiguate by using `local f3` to suppress this warning or `global f3` to assign to the existing global variable.
└ @ nothing In[38]:7
┌ Warning: Assignment to `Λ` in soft scope is ambiguous because a global variable by the same name exists: `Λ` will be treated as a new local. Disambiguate by using `local Λ` to suppress this warning or `global Λ` to assign to the existing global variable.
└ @ nothing In[38]:8
┌ Warning: Assignment to `K` in soft scope is ambiguous because a global variable by the same name exists: `K` will be treated as a new local. Disambiguate by usi

### Size 1 of 7
Rate 1 of 1


In [ ]:
t_minutes = sum(t) / 60
t_hours = sum(t) / 3600
d_avg = reduce(vcat, [mean.(filter.(!isnan, d[i])) for i in eachindex(ns)])
d_std = reduce(vcat, [std.(filter.(!isnan, d[i])) ./ length.(filter.(!isnan, d[i])) for i in eachindex(ns)]);

In [ ]:
# least squares
using DataFrames, GLM
ns_inv = 1 ./ ns
x = ns_inv
y = d_avg
data = DataFrame(X=x, Y=y)
ols = lm(@formula(Y ~ X), data)
intercept_stderr = stderror(ols)[1];
# xaug = [x ones(length(x))]
# (m, q) = xaug \ y

In [ ]:
q, m = coef(ols)

In [11]:
# compare w/ rsb
include("../code/rsb.jl")
using StatsBase, OffsetArrays
popsize = 2*10^6
J = 20
popP = fill(1/(2J+1), -J:J, 1:popsize)
popQ = fill(1/(2J+1), -J:J, 1:popsize)
R = Rs[1]
f3 = 1-3R
Λ = OffsetVector([0,0,1-f3,f3], 0:3)
K = OffsetVector([0,0,0,1], 0:3)
popP, popQ = RSB(Λ,K; J=J, maxiter=10^2, popsize=popsize, y=0.5, popP=popP, popQ=popQ);

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/crotti/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Progress: 100%|█████████████████████████████████████████| Time: 0:08:22
  err:  0.00011711449947417185/3.535533905932737e-5


err = 0.00011711449947417185


In [12]:
Λ = OffsetVector([0,0,1-f3,f3], 0:3)
K = OffsetVector([0,0,0,1], 0:3)
nsamples = 2*10^6
o_rsb_avg, _, _, o_rsb_std, _, _ = overlap1RSB_errorbars(Λ,K; popP=popP, popQ=popQ, y=0.5, samples=nsamples)
d_rsb_avg = 1/2*(1 - o_rsb_avg)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/crotti/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Progress: 100%|█████████████████████████████████████████| Time: 0:01:30
  F:  -0.5356130867990703
  O:  0.5327811992840774
  D:  0.2336094003579613


0.2331298905106048

In [13]:
d_rsb_std = 1/2 * o_rsb_std / sqrt(nsamples)

8.268077078276105e-7

In [ ]:
println("Runtime: ", round(t_hours,digits=1), " hours")
pl = plot()
for (i,R) in enumerate(Rs)
    pl = plot!(pl, 1 ./ ns, [d_avg[j][i] for j in eachindex(d_avg)], label="R=$R",
        m=:o, yerr=[d_std[j][i] for j in eachindex(d_std)], c=:blue, ms=3)
end
plot!(pl, xlabel="1/n", ylabel="distortion")
xrg = 0:1e-3:0.02
plot!(pl, xrg, xrg*m .+ q, label="linear fit", line=:dash, legend=:topleft)
scatter!(pl, [0], [q], m=:circle, c=:white, msc=:red, msw=1.5, label="prediction for n->inf",
    yerr=[intercept_stderr])
hline!(pl, [d_rsb_avg], label="RSB y=0.5", c=:green, ylims=(0.23, 0.26), ribbon=d_rsb_std)
pl2 = plot_rdb()
scatter!(pl2, R*ones(length(ns)), d_avg, label="exact enum", ms=2, xlims=(0.2, 0.3), ylims=(0.23,0.27), c=:blue)
scatter!(pl2, [R], [q], m=:circle, c=:white, msc=:red, msw=1.5, label="prediction for n->inf",
    yerr=[intercept_stderr])
scatter!(pl2, [R], [d_rsb_avg], c=:green, label="RSB y=0.5",m=:x, ms=3, yerr=[d_rsb_std])
plot(pl, pl2, size=(1000,400), margin=5Plots.mm)